In [1]:
import numpy as np
import scipy as sp

In [2]:
def rodrigez(fi,p):
    e=np.eye(3)
    px=np.array([[0,-p[2],p[1]],
                 [p[2],0,-p[0]],
                 [-p[1],p[0],0]])
    ppT=np.array([[p[0]**2,p[0]*p[1],p[0]*p[2]]
                 ,[p[1]*p[0],p[1]**2,p[1]*p[2]]
                 ,[p[2]*p[0],p[2]*p[1],p[2]**2]])
    return (1-np.cos(fi))*ppT+np.cos(fi)*e+np.sin(fi)*px

--------------------Odavde pocinje cetvrti domaci--------------------

In [51]:
import numpy as np
import scipy as sp
import math

class Quaternion:
    def __init__(self,params,w):
        self.i=params[0]
        self.j=params[1]
        self.k=params[2]
        self.params=params
        self.w=w
        
    def __truediv__(self,o:Quaternion)->Quaternion:
        if isinstance(o,Quaternion):
            donji=o.i**2+o.j**2+o.k**2+o.w**2
            return Quaternion([(o.w*self.i-o.i*self.w-o.j*self.k+o.k*self.j)/donji,
                              (o.w*self.j-o.i*self.k-o.j*self.w+o.k*self.i)/donji,
                              (o.w*self.k-o.i*self.j-o.j*self.i+o.k*self.w)/donji],
                              (o.w*self.w-o.i*self.i-o.j*self.j+o.k*self.k)/donji)
        else:
            a=[]
            for x in self.params:
                a.append(x/o)
            return Quaternion(a,self.w/o)
            
    def __add__(self,o:Quaternion)->Quaternion:
        if isinstance(o,Quaternion):
            return Quaternion([self.i+o.i,self.j+o.j,self.k+o.k],self.w+o.w)
        else:
            return Quaternion(self.params,self.w+o)
            
    def __radd__(self,o):
        if not isinstance(o,Quaternion) and not isinstance(o,str):
            return self.__add__(o)

    def __sub__(self,o:Quaternion)->Quaternion:
        if isinstance(o,Quaternion):
            return Quaternion([self.i-o.i,self.j-o.j,self.k-o.k],self.w-o.w)       
        else:
            return Quaternion(self.params,self.w-o)
            
    def __mul__(self,o:Quaternion)->Quaternion:
        # if self.w == 0 and o.w == 0 :
        #     novi_params=np.cross(self.params,o.params)
        #     novo_w=np.dot(self.params,o.params)
        #     return Quaternion([novi_params[0],
        #                       novi_params[1],
        #                       novi_params[2]],
        #                       novo_w)
        # else:
        #     novi_params=np.cross(self.params,o.params)+self.w*o.params+o.w*self.params
        #     novo_w=self.w*o.w-np.dot(self.params,o.params)
        #     return Quaternion([novi_params[0],
        #                       novi_params[1],
        #                       novi_params[2]],
        #                       novo_w)
        if isinstance(o,Quaternion): 
            return Quaternion([self.w*o.i+self.i*o.w+self.j*o.k-self.k*o.j,
                               self.w*o.j+self.j*o.w+self.k*o.i-self.i*o.k,
                               self.w*o.k+self.k*o.w+self.i*o.j-self.j*o.i],
                               self.w*o.w-self.i*o.i-self.j*o.j-self.k*o.k)
        else:
            a=[]
            for x in self.params:
                a.append(x*o)
            return Quaternion(a,self.w*o)

    def __rmul__(self,o):
        if not isinstance(o,Quaternion) and not isinstance(o,str):
            return self.__mul__(o)
            
    def __imul__(self,o:Quaternion)->Quaternion:
            return self.__mul__(o)
    def __idiv__(self,o:Quaternion)->Quaternion:
            return self.__truediv__(o)
        
    def konjugat(self)->Quaternion:
        a=[]
        for x in self.params:
            a.append(-x)
        return Quaternion(a,self.w)
        
    def norm(self)->float:
        return np.sqrt(self.i**2+self.j**2+self.k**2+self.w**2)
        
    def dot(self,o:Quaternion)->float:
        return self.i*o.i+self.j*o.j+self.k*o.k+self.w*o.w
        
    def inverse(self)->Quaternion:
        q=(self.konjugat()/(self.norm()**2)) 
        return Quaternion(q.params,q.w)
        
    def __str__(self)->str:
        return '['+str(self.i)+'i'+','+str(self.j)+'j'+','+str(self.k)+'k'+','+str(self.w)+']' 
    def __repr__(self)->str:
        return '['+str(self.i)+'i'+','+str(self.j)+'j'+','+str(self.k)+'k'+','+str(self.w)+']'

    def __neg__(self)->Quaternion:
        fi,p=Q2AngleAxis(self)
        return AngleAxis2Q(-fi,-p)

    def __eq__(self,o:Quaternion)->bool:
        return (self.i==o.i and self.j==o.j and self.k==o.k and self.w==o.w)

#dodato zbog predstavljanja SLerpa pomocu stepena (videcemo da li radi)
    def vector_part(self)->Quaternion:
        return (self-self.konjugat())/2
    def sign(self)->Quaternion:
        return self/self.norm()
    def arg(self)->float:
        return np.arccos(w/self.norm())
    def exp(self):
        return (self.vector_part().sign()*np.sin(self.vector_part().norm())+np.cos(self.vector_part().norm()))*math.exp(self.w)
    def ln(self):
        return self.sign()*np.sin(self.vector_part().norm())+math.log(self.norm())
    def __pow__(self,o:int):
        return (self.ln()*o).exp()

In [24]:
def AngleAxis2Q(fi,p):
    w=np.cos(fi/2)
    p/=np.linalg.norm(p)
    params=np.sin(fi/2)*p
    return Quaternion(params,w)

In [28]:
def Q2AngleAxis(q:Quaternion):
    q/=q.norm()
    if q.w<0:
        q=q*(-1)
    fi=2*np.arccos(q.w)
    if abs(q.w) == 1:
        p=np.array[1,0,0]
    else:
        p=q.params
        p/=np.linalg.norm(p)
    return fi,p

In [55]:
def Konjugacija(p:Quaternion,q:Quaternion)->Quaternion:
    #Cq(p)=qpq^-1
    return q*p*q.inverse()

In [32]:
def SLerp(q1:Quaternion,q2:Quaternion,tm:int):
    cos0=q1.dot(q2)
    #print(cos0)
    if cos0 < 0:
        q1=q1*(-1)
        cos0=-cos0
    if cos0 > 0.95:
        q1=q1*(-1) #ili lerp interpolaciju
    fi=np.arccos(cos0)
    return [q1*(np.sin(fi*(1-i/tm))/np.sin(fi))
            +q2*(np.sin(fi*(i/tm))/np.sin(fi)) for i in range(tm)]

In [61]:
q1=Quaternion([3,0,-5],1)
q2=Quaternion([1,1,0],7)

In [62]:
q7=Quaternion([1,0,2],2)

In [63]:
Konjugacija(q5,q6)

[0.0i,-1.9999999999999996j,-1.9999999999999996k,0.9999999999999997]